## Simple example reconstructing the sin function


## Imports and dataset

In [1]:
import HPO

import pandas as pd

import pysgpp

import sys

import math
import matplotlib.pyplot as plt
import torch

from sklearn.model_selection import cross_val_score

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers as opt
from scikeras.wrappers import KerasRegressor, KerasClassifier

import sklearn.metrics

def to_standard(lower, upper, value):
    return (value-lower)/(upper-lower)


def from_standard(lower, upper, value):
    return value*(upper-lower)+lower


dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
print(dataset.shape)
# separate the data from the target attributes
X = dataset[:,0:7]
Y = dataset[:,8]

dataset = HPO.Dataset(X, Y)


2023-02-28 18:02:23.439203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 18:02:24.873689: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/maxi/Repos/SGpp/lib:
2023-02-28 18:02:24.873810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/maxi/Repos/SGpp/lib:
2023-02-28 18:02:24.873817: W tensorflow/compiler/tf2tensorrt/utils/py_utils

(768, 9)


## Hyperparameterspace & Model & Function

In [2]:
BUDGET = 100
VERBOSE = 2
CV = 2
SCORING = 'accuracy'

hyperparameterspace = {
    'model__loss': ["list", 'binary_crossentropy', 'categorical_crossentropy', 'binary_crossentropy', 'mean_squared_error', 'mean_absolute_error'],
    'model__optimizer': ["list", 'sgd', 'rmsprop', 'adam', 'adadelta', 'adagrad', 'adamax', 'adafactor', 'nadam', 'ftrl'],
    'epochs': ["interval-int", 1, 400],
    'optimizer__learning_rate': ["interval", 0.000001, 0.001]
}

##################### Model for grid and random search #####################

# Function to create model, required for KerasClassifier
def create_model(optimizer, loss):
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(7,), activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    if optimizer == 'sgd':
        optimizer = opt.SGD()
    elif optimizer == 'rmsprop':
        optimizer = opt.RMSprop()
    elif optimizer == 'adam':
        optimizer = opt.Adam()
    elif optimizer == 'adadelta':
        optimizer = opt.Adadelta()
    elif optimizer == 'adagrad':
        optimizer = opt.Adagrad()
    elif optimizer == 'adamax':
        optimizer = opt.Adamax()
    elif optimizer == 'adafactor':
        optimizer = opt.Adafactor()
    elif optimizer == 'nadam':
        optimizer = opt.Nadam()
    elif optimizer == 'ftrl':
        optimizer = opt.Ftrl()

    # Compile model
    model.compile(loss=loss,
                  optimizer=optimizer)
    return model

model = KerasClassifier(model=create_model, verbose=0)

##################### Blackbox function for bayesian optimization #####################

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste


def blackboxfunction(params):  # Maximizing 
    index = int(params[0]*(len(hyperparameterspace_special["model__loss"])-1))
    loss = hyperparameterspace_special["model__loss"][index]
    
    index = int(params[1]*(len(hyperparameterspace_special["model__optimizer"])-1))
    model_optimizer = hyperparameterspace_special["model__optimizer"][index]

    epochs = int(params[2])

    learning_rate = params[3]

    # Function to create model, required for KerasClassifier
    def create_model():
        # create model
        model = Sequential()
        model.add(Dense(10, input_shape=(7,), activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        if model_optimizer == 'sgd':
            optimizer = opt.SGD(learning_rate=learning_rate)
        elif model_optimizer == 'rmsprop':
            optimizer = opt.RMSprop(learning_rate=learning_rate)
        elif model_optimizer == 'adam':
            optimizer = opt.Adam(learning_rate=learning_rate)
        elif model_optimizer == 'adadelta':
            optimizer = opt.Adadelta(learning_rate=learning_rate)
        elif model_optimizer == 'adagrad':
            optimizer = opt.Adagrad(learning_rate=learning_rate)
        elif model_optimizer == 'adamax':
            optimizer = opt.Adamax(learning_rate=learning_rate)
        elif model_optimizer == 'nadam':
            optimizer = opt.Nadam(learning_rate=learning_rate)
        elif model_optimizer == 'ftrl':
            optimizer = opt.Ftrl(learning_rate=learning_rate)

        # Compile model
        model.compile(loss=loss,
                    optimizer=optimizer)
        return model

    model = KerasClassifier(model=create_model, verbose=0)


    model.fit(dataset.get_X_train(), dataset.get_Y_train(), epochs=epochs)

    Y_predicted = model.predict(dataset.get_X_test())

    return sklearn.metrics.accuracy_score(dataset.get_Y_test(), Y_predicted)
    

##################### Function for sparse grid search #####################
class ExampleFunction(pysgpp.ScalarFunction): # Minimizing

    def __init__(self):
        super(ExampleFunction, self).__init__(len(hyperparameterspace))


    def eval(self, x):
        index = int(x[0]*(len(hyperparameterspace_special["model__loss"])-1))
        loss = hyperparameterspace_special["model__loss"][index]
        
        index = int(x[1]*(len(hyperparameterspace_special["model__optimizer"])-1))
        model_optimizer = hyperparameterspace_special["model__optimizer"][index]
        
        
        epochs = int(from_standard(1, 400, x[2]))

        learning_rate = from_standard(0.000001, 0.001, x[3])
        
        # Function to create model, required for KerasClassifier
        def create_model():
            # create model
            model = Sequential()
            model.add(Dense(10, input_shape=(7,), activation='relu'))
            model.add(Dense(20, activation='relu'))
            model.add(Dense(10, activation='relu'))
            model.add(Dense(20, activation='relu'))
            model.add(Dense(10, activation='relu'))
            model.add(Dense(1, activation='sigmoid'))

            if model_optimizer == 'sgd':
                optimizer = opt.SGD(learning_rate=learning_rate)
            elif model_optimizer == 'rmsprop':
                optimizer = opt.RMSprop(learning_rate=learning_rate)
            elif model_optimizer == 'adam':
                optimizer = opt.Adam(learning_rate=learning_rate)
            elif model_optimizer == 'adadelta':
                optimizer = opt.Adadelta(learning_rate=learning_rate)
            elif model_optimizer == 'adagrad':
                optimizer = opt.Adagrad(learning_rate=learning_rate)
            elif model_optimizer == 'adamax':
                optimizer = opt.Adamax(learning_rate=learning_rate)
            elif model_optimizer == 'adafactor':
                optimizer = opt.Adafactor(learning_rate=learning_rate)
            elif model_optimizer == 'nadam':
                optimizer = opt.Nadam(learning_rate=learning_rate)
            elif model_optimizer == 'ftrl':
                optimizer = opt.Ftrl(learning_rate=learning_rate)

            # Compile model
            model.compile(loss=loss,
                        optimizer=optimizer)
            return model

        model = KerasClassifier(model=create_model, verbose=0)


        model.fit(dataset.get_X_train(), dataset.get_Y_train(), epochs=epochs)

        Y_predicted = model.predict(dataset.get_X_test())

        return -sklearn.metrics.accuracy_score(dataset.get_Y_test().tolist(), Y_predicted)

## Grid search

In [3]:
optimization = HPO.Optimization(dataset, model, hyperparameterspace, type="grid_search", cv=CV, scoring=SCORING, budget=BUDGET, verbosity=VERBOSE)
result = optimization.fit()

print("Best result:")
print(result.best_score_)
print("Corresponding parameters:")
print(result.best_params_)

Fitting 2 folds for each of 45 candidates, totalling 90 fits


2023-02-28 18:02:27.723215: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-28 18:02:27.723309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (maxi-XPS-15-9560): /proc/driver/nvidia/version does not exist
2023-02-28 18:02:27.725146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=sgd, optimizer__learning_rate=0.0005005000000000001; total time=   6.2s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=sgd, optimizer__learning_rate=0.0005005000000000001; total time=   5.6s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=rmsprop, optimizer__learning_rate=0.0005005000000000001; total time=   5.1s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=rmsprop, optimizer__learning_rate=0.0005005000000000001; total time=   5.4s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=adam, optimizer__learning_rate=0.0005005000000000001; total time=   6.3s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=adam, optimizer__learning_rate=0.0005005000000000001; total time=   5.7s
[CV] END epochs=200, model__loss=binary_crossentropy, model__optimizer=adadelta, optimizer__learning_rate=0.0005005000000000001; tot

KeyboardInterrupt: 

## Random search

In [4]:
optimization = HPO.Optimization(dataset, model, hyperparameterspace, type="random_search", cv=CV, scoring=SCORING, budget=BUDGET, verbosity=VERBOSE)
result = optimization.fit()
print("Best result:")
print(result.best_score_)
print("Corresponding parameters:")
print(result.best_params_)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV] END epochs=200, model__loss=mean_squared_error, model__optimizer=adafactor, optimizer__learning_rate=0.0005005000000000001; total time=   0.1s
[CV] END epochs=200, model__loss=mean_squared_error, model__optimizer=adafactor, optimizer__learning_rate=0.0005005000000000001; total time=   0.1s
[CV] END epochs=200, model__loss=mean_absolute_error, model__optimizer=adam, optimizer__learning_rate=0.0005005000000000001; total time=   5.5s


KeyboardInterrupt: 

## Bayesian Optimization 

https://thuijskens.github.io/2016/12/29/bayesian-optimisation/ 

In [5]:
optimization = HPO.Optimization(dataset, blackboxfunction, hyperparameterspace, type="bayesian", budget=BUDGET, verbosity=VERBOSE)
result = optimization.fit()

# fig = plt.figure()
# ax = plt.axes(projection='3d')

# x = [result[0][i][0] for i in range(len(result[0]))]
# y = [result[0][i][1] for i in range(len(result[0]))]

# ax.scatter3D(x, y, result[1], c=result[1], cmap='Greens')

index_best = 0
for i in range(len(result[1])):
    if result[1][i] == max(result[1]):
        index_best = i

best_score = result[1][index_best]
best_params = result[0][index_best]


print("Best MSE:")
print(best_score)
print("With Hyperparameters: ")
i = 0
for key in hyperparameterspace.keys():
    if hyperparameterspace[key][0] == "list":
        index = int(best_params[i]*(len(hyperparameterspace_special[key])-1))
        print(key + ": " + str(hyperparameterspace_special[key][index]))
    else:
        print(key + ": " + str(best_params[i]))
    i += 1


print([result[0][i] for i in range(len(result[0])) if result[1][i] == max(result[1])])
#print(result)

Progress: [####----------------] 20.0%
Current time per iteration: 17.58371454999724
Approx. time remaining: 70.33485819998896


KeyboardInterrupt: 

## Sparse grid search

In [6]:
f = ExampleFunction()

optimization = HPO.Optimization(dataset, f, hyperparameterspace, type="sparse", budget=BUDGET, verbosity=VERBOSE)
result = optimization.fit()


Adaptive grid generation (Ritter-Novak)...
